In [2]:
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.14.1-alpha"
#r "nuget: dotenv.net"

Installed Packages dotenv.net, 3.2.1 Microsoft.SemanticKernel.Agents.Abstractions, 1.14.1-alpha Microsoft.SemanticKernel.Agents.Core, 1.14.1-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.14.1-alpha Microsoft.SemanticKernel.Connectors.OpenAI, 1.14.1

In [5]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents.Chat;
using dotenv.net;
using dotenv.net.Utilities;

In [7]:
DotEnv.Load();
var modelDeploymentName =EnvReader.GetStringValue("deploymentName");
var modelApiKey =EnvReader.GetStringValue("apiKey");
var modelEndPoint=EnvReader.GetStringValue("endpoint");


In [8]:
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddAzureOpenAIChatCompletion(
    deploymentName: modelDeploymentName,
    apiKey: modelApiKey,
    endpoint: modelEndPoint
    //modelId: "gpt-4o", // Optional name of the underlying model if the deployment name doesn't match the model name
    //serviceId: "YOUR_SERVICE_ID" // Optional; for targeting specific services within Semantic Kernel
);
Kernel kernel = kernelBuilder.Build();

In [9]:
string ProgramManager = """
    You are a program manager which will take the requirement and create a plan for creating app. Program Manager understands the 
    user requirements and form the detail documents with requirements and costing. 
""";

string SoftwareEngineer = """
   You are Software Engieer, and your goal is develop web app using HTML and JavaScript (JS) by taking into consideration all
   the requirements given by Program Manager. 
""";

string Manager = """
    You are manager which will review software engineer code, and make sure all client requirements are completed.
     Once all client requirements are completed, you can approve the request by just responding "approve"
""";

#pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details

 ChatCompletionAgent ProgramManagerAgent =
            new()
            {
                Instructions = ProgramManager,
                Name = "ProgaramManagerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent SoftwareEngineerAgent =
            new()
            {
                Instructions = SoftwareEngineer,
                Name = "SoftwareEngineerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent ProjectManagerAgent =
            new()
            {
                Instructions = Manager,
                Name = "ProjectManagerAgent",
                Kernel = kernel
            };

In [10]:
#pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details

using System.Threading;

private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
}

In [11]:
#pragma warning disable SKEXP0110, SKEXP0001 

AgentGroupChat chat =
    new(ProgramManagerAgent, SoftwareEngineerAgent, ProjectManagerAgent)
    {
        ExecutionSettings =
            new()
            {
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        Agents = [ProjectManagerAgent],
                        MaximumIterations = 6,
                    }
            }
    };

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 
// Invoke chat and display messages.
string input = """

I want to develop app which will provide me calculator. Keep it very simple. And get final approval from manager.
""";

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

# user: '
I want to develop app which will provide me calculator. Keep it very simple. And get final approval from manager.'
